# CODE AI ASSISTANT

In [2]:
#importing

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.models import PointStruct

import openai
import requests
import json

In [3]:
OPENAI_API_KEYS = "YOUR OPEN AI KEYS" #put your own openAI keys

embedding_model = "text-embedding-3-small"
assistant_model = "gpt-3.5-turbo"

# Getting Article from Strapi

In [4]:
r = requests.get(url ="http://localhost:1338/api/reviews") #put URL for your API 
articles = r.json()

articles['data'][0]

{'id': 1,
 'attributes': {'title': 'Star Wars: The Empire Strikes Back',
  'rating': 8,
  'body': 'It\'s generally agreed that The Empire Strikes Back is the best film of George Lucas\' initial trilogy (despite a latter-day shift toward the original\'s storytelling purity). Not a sequel as such, but the next part of a continuing story, Empire marks enormous progression both in terms of the mythos of the series and in the filmmaking quality itself.\n\nNo longer tethered by the need to establish this fabulous universe wrapped in the arcane mysticism of the Force, this is a film far more sophisticated, awe-inspiring and daring (what do you mean Han Solo stays frozen in carbonite?). The actors too, reassured this was not some tinpot sci-fi quickie, have settled comfortably into their characters. Which is a good thing given the nightmare wrought for them by writers Lucas, Lawrence Kasdan and Leigh Brackett. At once more graceful and melancholic than its predecessor, Kershner enhances the pe

# OpenAI and Qdrant

In [5]:
openai_client = openai.Client(
    api_key= OPENAI_API_KEYS 
)

client = QdrantClient(host="localhost", port=6333) #your own client

In [6]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='reviews_collection')])

Embedding the data

In [7]:
def convert_embedding(texts):
    result = openai_client.embeddings.create(input=texts, model=embedding_model)
    return result

In [8]:
#payload data
payload_data = []
for article in articles['data']:
    payload_data.append({
        'title' : convert_embedding(article['attributes']['title']).data[0].embedding,
        'body' : convert_embedding(article['attributes']['body']).data[0].embedding,
    })
payload_data[0]

{'title': [-0.034626979380846024,
  0.06731407344341278,
  -0.04852141812443733,
  -0.027546392753720284,
  -0.03714883327484131,
  0.0365426167845726,
  -0.005819660611450672,
  0.04071337357163429,
  -0.0010487512918189168,
  0.02410309389233589,
  0.006735044531524181,
  -0.0009767633164301515,
  -0.016076812520623207,
  -0.0394524484872818,
  0.025824742391705513,
  -0.009226586669683456,
  0.0017383204540237784,
  -0.009590315632522106,
  -0.018089445307850838,
  0.03414200618863106,
  -0.005671137943863869,
  0.010705750435590744,
  0.006498620845377445,
  0.03455423563718796,
  -0.010972484946250916,
  0.01922912709414959,
  -0.032153625041246414,
  -0.0475514754652977,
  0.03208087757229805,
  -0.02016269788146019,
  0.026940178126096725,
  -0.020405184477567673,
  0.043429214507341385,
  -0.03511195257306099,
  0.0036979091819375753,
  0.004325341433286667,
  -0.017143750563263893,
  -0.02497604303061962,
  -0.011372586712241173,
  -0.016501162201166153,
  0.02975301444530487,

Indexing

In [11]:
collection_name = "reviews_collection"

In [42]:
# Create Qdrant collection
#ALREADY CREATED
vector_size = len(payload_data[0]['body'])
client.create_collection(
    collection_name='reviews_collection',
    vectors_config={
        'title': VectorParams(
            distance=Distance.COSINE,
            size=vector_size,
        ),
        'content': VectorParams(
            distance=Distance.COSINE,
            size=vector_size,
        ),
    }
)

True

In [ ]:
#create points

In [12]:
points = [
    PointStruct(
        id=idx,
        vector={
            'title': data['title'], 
            'content': data['body']
        },
        payload={
            'title': text['attributes']['title'],
            'rating': text['attributes']['rating'],
            'content': text['attributes']['body']
        }
    )
    for idx, (data, text) in enumerate(zip(payload_data, articles['data']))
]

In [13]:
points[0]

PointStruct(id=0, vector={'title': [-0.034626979380846024, 0.06731407344341278, -0.04852141812443733, -0.027546392753720284, -0.03714883327484131, 0.0365426167845726, -0.005819660611450672, 0.04071337357163429, -0.0010487512918189168, 0.02410309389233589, 0.006735044531524181, -0.0009767633164301515, -0.016076812520623207, -0.0394524484872818, 0.025824742391705513, -0.009226586669683456, 0.0017383204540237784, -0.009590315632522106, -0.018089445307850838, 0.03414200618863106, -0.005671137943863869, 0.010705750435590744, 0.006498620845377445, 0.03455423563718796, -0.010972484946250916, 0.01922912709414959, -0.032153625041246414, -0.0475514754652977, 0.03208087757229805, -0.02016269788146019, 0.026940178126096725, -0.020405184477567673, 0.043429214507341385, -0.03511195257306099, 0.0036979091819375753, 0.004325341433286667, -0.017143750563263893, -0.02497604303061962, -0.011372586712241173, -0.016501162201166153, 0.02975301444530487, -0.04003441333770752, -0.0033553980756551027, -0.02504

In [46]:
#inserting article into Qdrant
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
client.count(collection_name=collection_name)

CountResult(count=7)

# Searching Content

In [22]:
def query_qdrant(query, collection_name, vector_name='title', top_k=3):
    # Creates embedding vector from user query
    embedded_query = convert_embedding(query).data[0].embedding # We take the first embedding from the list
    
    query_results = client.search(
        collection_name=collection_name,
        query_vector=(
            vector_name, embedded_query
        ),
        limit=top_k, 
        query_filter=None
    )
    
    return query_results

In [16]:
query_results = query_qdrant('Super Hero Movie recommendation', collection_name, 'title')
for i, article in enumerate(query_results):
    print(f'{i + 1}. {article.payload["title"]}')

1. Black Panther
2. Avengers: Endgame
3. The Dark Knight


In [154]:
query_results = query_qdrant('horror movie', collection_name, 'title')
for i, article in enumerate(query_results):
    print(f'{i + 1}. {article.payload["title"]}')

1. The Dark Knight
2. Get Out
3. The Godfather


# Completion API OPENAI

In [143]:
functions = [
  {
    "type": "function",
    "function": {
      "name": "get_movie_recommendation",
      "description": "Get the recommendation of the movie from the user",
      "parameters": {
        "type": "object",
        "properties": {
          "genre": {
            "type": "string",
            "description": "The genre of the movie, e.g. superhero, romance,horror",
          },
          "total": {
            "type": "string",
            "description": "The number of recommendation the user want, e.g. 2,5,8",
          },
        },
        "required": ["genre","total"],
      },
    }
  }
]

In [156]:
# function call
def get_movie_recommendation(genre,total):
    total = int(total)
    
    query_results = query_qdrant(genre, collection_name, 'title',total)
    list_str = ""
    for article in query_results:
        list_str += article.payload["title"]
        list_str += "   "
    
    result ={
        "list":  list_str,
        "total": str(total)
    }
    return json.dumps(result)

In [157]:
def first_completion(text_input):
    completion = openai_client.chat.completions.create(
      model=assistant_model,
      messages=[
          {"role": "system", "content": "You are a helpful assistant who knows a lot in the movie industry."},
          {"role": "user", "content": text_input},
      ],
      tools=functions,
      tool_choice="auto"
    )
    output = completion.choices[0].message
    return output


In [164]:
def second_completion(name, content,text_input):
    second_completion = openai_client.chat.completions.create(
      model= assistant_model,
      messages= [
          {"role": "user", "content": text_input},
          {"role": "function", "name": name, "content": content},
      ],
      tools=functions,
    )
    #print(second_completion)
    output = second_completion.choices[0].message.content
    return output

In [163]:
user_prompt = ""
while(user_prompt != "quit"):
    user_prompt = str(input())
    if(user_prompt=="quit"):
       break
    output = first_completion(user_prompt)
    
    
    params = json.loads(output.tool_calls[0].function.arguments)
    chosen_function = eval(output.tool_calls[0].function.name)
    movie = chosen_function(**params)
    #print(params)
    #print(movie)

    #print(output.tool_calls[0].function.name)
    response = second_completion(output.tool_calls[0].function.name,movie,user_prompt)
    print("-----------------------------AI RESPONSE--------------------------------------")
    print(response)

 give me 3 recomendation about superhero movie


{'genre': 'superhero', 'total': '3'}
{"list": "Black Panther   Avengers: Endgame   The Dark Knight   ", "total": "3"}
get_movie_recommendation
ChatCompletion(id='chatcmpl-9aXGj6wTHZiXTMa7Ml2251V5QFM6B', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I have 3 superhero movie recommendations for you:\n1. Black Panther\n2. Avengers: Endgame\n3. The Dark Knight', role='assistant', function_call=None, tool_calls=None))], created=1718494721, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=130, total_tokens=158))
-----------------------------AI RESPONSE--------------------------------------
I have 3 superhero movie recommendations for you:
1. Black Panther
2. Avengers: Endgame
3. The Dark Knight


 "give me 2 recomendation about superhero movie"


{'genre': 'superhero', 'total': '2'}
{"list": "Black Panther   Avengers: Endgame   ", "total": "2"}
get_movie_recommendation
ChatCompletion(id='chatcmpl-9aXGwJZmoczN80oyKNuwirv0TTcHU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are 2 recommendations for superhero movies:\n1. Black Panther\n2. Avengers: Endgame\n\nI hope you enjoy watching them!', role='assistant', function_call=None, tool_calls=None))], created=1718494734, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=128, total_tokens=157))
-----------------------------AI RESPONSE--------------------------------------
Here are 2 recommendations for superhero movies:
1. Black Panther
2. Avengers: Endgame

I hope you enjoy watching them!


 "give me 1 recomendation about action movie"


{'genre': 'action', 'total': '1'}
{"list": "Get Out   ", "total": "1"}
get_movie_recommendation
ChatCompletion(id='chatcmpl-9aXHAPiTb3FZ1NpIKFS0lEdnHvPNz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I recommend you watch "Get Out" for an action movie!', role='assistant', function_call=None, tool_calls=None))], created=1718494748, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=123, total_tokens=137))
-----------------------------AI RESPONSE--------------------------------------
I recommend you watch "Get Out" for an action movie!


 quit


second_completion = openai_client.chat.completions.create(
  model=assistant_model,
  messages=[
    {"role": "user", "content": "give me 3 recomendation about superhero movie"},
    {"role": "function", "name": output.tool_calls[0].function.name, "content": movie},
  ],
  tools=functions,
)

foutput = second_completion.choices[0].message.content
print(foutput)